<a href="https://colab.research.google.com/github/duanmenghuan/Car_CRUD/blob/master/ONNX.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import models

# 有 GPU 就用 GPU，没有就用 CPU
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


In [ ]:
model = models.resnet18(pretrained=True)
model = model.eval().to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 102MB/s]


In [ ]:
x = torch.randn(1, 3, 256, 256).to(device)

In [ ]:
output = model(x)

In [ ]:
output.shape

torch.Size([1, 1000])

In [ ]:
!pip install onnx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 68.9 MB/s eta 0:00:00


In [ ]:
with torch.no_grad():
    torch.onnx.export(
        model,                       # 要转换的模型
        x,                           # 模型的任意一组输入
        'resnet18_imagenet.onnx',    # 导出的 ONNX 文件名
        opset_version=11,            # ONNX 算子集版本
        input_names=['input'],       # 输入 Tensor 的名称
        output_names=['output']      # 输出 Tensor 的名称
    )

In [ ]:
import onnx

# 读取 ONNX 模型
onnx_model = onnx.load('resnet18_imagenet.onnx')

# 检查模型格式是否正确
onnx.checker.check_model(onnx_model)

print('无报错，onnx模型载入成功')

无报错，onnx模型载入成功


In [ ]:
print(onnx.helper.printable_graph(onnx_model.graph))

graph main_graph (
  %input[FLOAT, 1x3x256x256]
) initializers (
  %fc.weight[FLOAT, 1000x512]
  %fc.bias[FLOAT, 1000]
  %onnx::Conv_193[FLOAT, 64x3x7x7]
  %onnx::Conv_194[FLOAT, 64]
  %onnx::Conv_196[FLOAT, 64x64x3x3]
  %onnx::Conv_197[FLOAT, 64]
  %onnx::Conv_199[FLOAT, 64x64x3x3]
  %onnx::Conv_200[FLOAT, 64]
  %onnx::Conv_202[FLOAT, 64x64x3x3]
  %onnx::Conv_203[FLOAT, 64]
  %onnx::Conv_205[FLOAT, 64x64x3x3]
  %onnx::Conv_206[FLOAT, 64]
  %onnx::Conv_208[FLOAT, 128x64x3x3]
  %onnx::Conv_209[FLOAT, 128]
  %onnx::Conv_211[FLOAT, 128x128x3x3]
  %onnx::Conv_212[FLOAT, 128]
  %onnx::Conv_214[FLOAT, 128x64x1x1]
  %onnx::Conv_215[FLOAT, 128]
  %onnx::Conv_217[FLOAT, 128x128x3x3]
  %onnx::Conv_218[FLOAT, 128]
  %onnx::Conv_220[FLOAT, 128x128x3x3]
  %onnx::Conv_221[FLOAT, 128]
  %onnx::Conv_223[FLOAT, 256x128x3x3]
  %onnx::Conv_224[FLOAT, 256]
  %onnx::Conv_226[FLOAT, 256x256x3x3]
  %onnx::Conv_227[FLOAT, 256]
  %onnx::Conv_229[FLOAT, 256x128x1x1]
  %onnx::Conv_230[FLOAT, 256]
  %onnx::Conv_2

In [ ]:
import torch

model = models.resnet18(pretrained=True)
model = model.eval().to(device)
x = torch.randn(1, 3, 224, 224).to(device)
onnx_path = 'resnet18_imagenet_224.onnx'
input_names = ['input']
output_names = ['output']

# 创建一个txt文件来存储对应关系
with open('tensor_names.txt', 'w') as f:
    for i, input_name in enumerate(input_names):
        f.write(f'Input Tensor {i}: {input_name}\n')
    for i, output_name in enumerate(output_names):
        f.write(f'Output Tensor {i}: {output_name}\n')

# 导出ONNX模型
with torch.no_grad():
    torch.onnx.export(
        model,
        x,
        onnx_path,
        opset_version=11,
        input_names=input_names,
        output_names=output_names
    )


In [ ]:
import onnxruntime
import numpy as np
import torch
import torch.nn.functional as F

import pandas as pd

In [ ]:
!pip install onnxruntime

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 11.3 MB/s eta 0:00:00


In [ ]:
import onnxruntime
import numpy as np
import torch
import torch.nn.functional as F

import pandas as pd

In [ ]:
ort_session = onnxruntime.InferenceSession('resnet18_imagenet.onnx')

In [ ]:
x = torch.randn(1, 3, 256, 256).numpy()

In [ ]:
x.shape

(1, 3, 256, 256)

In [ ]:
# onnx runtime 输入
ort_inputs = {'input': x}

# onnx runtime 输出
ort_output = ort_session.run(['output'], ort_inputs)[0]

In [ ]:
ort_output.shape

(1, 1000)

In [ ]:
ort_output

array([[ 2.08065867e-01,  2.30353332e+00,  2.40817595e+00,
         2.57714820e+00,  3.91723084e+00,  3.90490294e+00,
         3.96908283e+00,  2.52534181e-01, -8.49148929e-01,
        -2.48722643e-01, -1.64122701e-01,  1.71915388e+00,
         9.82266188e-01,  1.98348653e+00,  2.34572959e+00,
         1.09311831e+00, -1.82192981e-01,  2.44818285e-01,
         1.68290663e+00,  1.09627032e+00,  3.72058541e-01,
         3.76914024e-01,  1.73917603e+00,  9.41701412e-01,
         5.03122509e-01,  1.70982718e-01,  1.11003757e+00,
         9.37931538e-01,  1.36173546e-01,  6.49981737e-01,
         9.55007195e-01,  5.46512425e-01, -7.92838573e-01,
         2.53557181e+00,  2.97743320e+00,  8.76599014e-01,
         3.00345957e-01,  4.47058797e-01,  5.46487570e-01,
         1.91566229e+00,  2.05188727e+00,  1.20644236e+00,
         1.50006628e+00,  5.72565019e-01,  1.80017328e+00,
         2.39068270e-03,  2.25543475e+00, -7.27296710e-01,
         1.32667542e+00,  4.58254933e-01,  1.67858696e+0

In [ ]:
!wget https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/imagenet/imagenet_class_index.csv

--2023-12-11 02:46:26--  https://zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com/20220716-mmclassification/dataset/imagenet/imagenet_class_index.csv
Resolving zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)... 121.36.235.132
Connecting to zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com (zihao-openmmlab.obs.cn-east-3.myhuaweicloud.com)|121.36.235.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37074 (36K) [text/csv]
Saving to: ‘imagenet_class_index.csv’

imagenet_class_inde 100%[===================>]  36.21K  --.-KB/s    in 0.09s   

2023-12-11 02:46:27 (412 KB/s) - ‘imagenet_class_index.csv’ saved [37074/37074]



In [ ]:
df = pd.read_csv('imagenet_class_index.csv')
idx_to_labels = {}
for idx, row in df.iterrows():
    #idx_to_labels[row['ID']] = row['class']   # 英文
    idx_to_labels[row['ID']] = row['Chinese'] # 中文

In [ ]:
idx_to_labels

{0: '丁鲷',
 1: '金鱼',
 2: '大白鲨',
 3: '虎鲨',
 4: '锤头鲨',
 5: '电鳐',
 6: '黄貂鱼',
 7: '公鸡',
 8: '母鸡',
 9: '鸵鸟',
 10: '燕雀',
 11: '金翅雀',
 12: '家朱雀',
 13: '灯芯草雀',
 14: '靛蓝雀,靛蓝鸟',
 15: '蓝鹀',
 16: '夜莺',
 17: '松鸦',
 18: '喜鹊',
 19: '山雀',
 20: '河鸟',
 21: '鸢（猛禽）',
 22: '秃头鹰',
 23: '秃鹫',
 24: '大灰猫头鹰',
 25: '欧洲火蝾螈',
 26: '普通蝾螈',
 27: '水蜥',
 28: '斑点蝾螈',
 29: '蝾螈,泥狗',
 30: '牛蛙',
 31: '树蛙',
 32: '尾蛙,铃蟾蜍,肋蟾蜍,尾蟾蜍',
 33: '红海龟',
 34: '皮革龟',
 35: '泥龟',
 36: '淡水龟',
 37: '箱龟',
 38: '带状壁虎',
 39: '普通鬣蜥',
 40: '美国变色龙',
 41: '鞭尾蜥蜴',
 42: '飞龙科蜥蜴',
 43: '褶边蜥蜴',
 44: '鳄鱼蜥蜴',
 45: '毒蜥',
 46: '绿蜥蜴',
 47: '非洲变色龙',
 48: '科莫多蜥蜴',
 49: '非洲鳄,尼罗河鳄鱼',
 50: '美国鳄鱼,鳄鱼',
 51: '三角龙',
 52: '雷蛇,蠕虫蛇',
 53: '环蛇,环颈蛇',
 54: '希腊蛇',
 55: '绿蛇,草蛇',
 56: '国王蛇',
 57: '袜带蛇,草蛇',
 58: '水蛇',
 59: '藤蛇',
 60: '夜蛇',
 61: '大蟒蛇',
 62: '岩石蟒蛇,岩蛇,蟒蛇',
 63: '印度眼镜蛇',
 64: '绿曼巴',
 65: '海蛇',
 66: '角腹蛇',
 67: '菱纹响尾蛇',
 68: '角响尾蛇',
 69: '三叶虫',
 70: '盲蜘蛛',
 71: '蝎子',
 72: '黑金花园蜘蛛',
 73: '谷仓蜘蛛',
 74: '花园蜘蛛',
 75: '黑寡妇蜘蛛',
 76: '狼蛛',
 77: '狼蜘蛛,狩猎蜘蛛',
 78: '壁虱',
 79: '

In [ ]:
with open('imagenet_class_index_chinese.txt', 'w', encoding='utf-8') as file:
    for idx, label in idx_to_labels.items():
        file.write(f'{idx}: {label}\n')